In [1]:
import pandas as pd
from IPython.display import Markdown, display
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent
import os

In [2]:
os.environ["OPENAI_API_KEY"] = open("API_KEY",'r').read()

In [3]:
data = pd.read_csv('titanic.csv')

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
data.shape

(891, 12)

In [6]:
chayGPTAgent = create_pandas_dataframe_agent(
    ChatOpenAI(model_name='gpt-4'),
    df=data,
    verbose=True
)

In [7]:
Markdown(chayGPTAgent.agent.llm_chain.prompt.template)   


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}

In [8]:
chayGPTAgent.run('How many rows does this dataset contain ?')



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataset, I can use the `shape` attribute of the dataframe.

Action: python_repl_ast
Action Input: df.shape[0]
Observation: 891
Thought:The dataset contains 891 rows. 
Final Answer: 891

> Finished chain.


'891'

In [10]:
prompt = ('Qui sont les passagers les plus vieux ? ')
chayGPTAgent.run(prompt)



> Entering new AgentExecutor chain...
Thought: To find the oldest passengers, we need to sort the dataframe by age in descending order.

Action: python_repl_ast
Action Input: df.sort_values('Age', ascending=False)
Observation:      PassengerId  Survived  Pclass                                      Name  \
630          631         1       1      Barkworth, Mr. Algernon Henry Wilson   
851          852         0       3                       Svensson, Mr. Johan   
493          494         0       1                   Artagaveytia, Mr. Ramon   
96            97         0       1                 Goldschmidt, Mr. George B   
116          117         0       3                      Connors, Mr. Patrick   
..           ...       ...     ...                                       ...   
859          860         0       3                          Razi, Mr. Raihed   
863          864         0       3         Sage, Miss. Dorothy Edith "Dolly"   
868          869         0       3               va

'The oldest passengers are:\n- Barkworth, Mr. Algernon Henry Wilson, aged 80\n- Svensson, Mr. Johan, aged 74\n- Artagaveytia, Mr. Ramon, aged 71\n- Goldschmidt, Mr. George B, aged 71\n- Connors, Mr. Patrick, aged 70.5'